<a href="https://colab.research.google.com/github/alex2844/youtube-translate/blob/main/ytranslate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Youtube Translate &nbsp; [![github][github_img]][github_url]
[gitHub_img]: https://img.shields.io/badge/GitHub-%23121011.svg?logo=github&logoColor=white
[github_url]: https://github.com/alex2844/youtube-translate

In [ ]:
# @title Settings
import os
import shutil
from google.colab import drive

TEMP_DIR = '/content/.ytranslate'
OUTPUT_DIR = '/content/videos'
DRIVE_MOUNT_PATH = '/content/drive'
CONFIG_PATH = os.path.join(DRIVE_MOUNT_PATH, 'MyDrive/ytranslate/config.env')

URL = '' # @param {"type":"string","placeholder":"YouTube URL or Google Drive Path"}
HEIGHT = 0 # @param {"type":"integer","placeholder":"Max height (0 = Best)"}
FROMLANG = 'en' # @param ["en", "ru", "zh", "ko", "ar", "fr", "it", "es", "de", "ja" ]
TOLANG = 'ru' # @param ["ru", "en", "kk" ]
SAVE = 'none' # @param ["save", "download", "none"]
COOKIES = '' # @param {"type":"string","placeholder":"Optional cookies path"}
FORCE_AVC = False # @param {"type":"boolean"}
MARK_WATCHED = False # @param {type:"boolean"}
ADD_INDEX = False # @param {type:"boolean"}
CLEAR_OUTPUT = True # @param {type:"boolean"}
USE_DRIVE_CONFIG = False # @param {type:"boolean"}

os.environ['YT_URL'] = URL
os.environ['YT_HEIGHT'] = f'{HEIGHT or ""}'
os.environ['YT_FROMLANG'] = FROMLANG
os.environ['YT_TOLANG'] = TOLANG
os.environ['YT_SAVE'] = SAVE
os.environ['YT_FORCE_AVC'] = 'true' if FORCE_AVC else 'false'
os.environ['YT_MARK_WATCHED'] = 'true' if MARK_WATCHED else 'false'
os.environ['YT_ADD_INDEX'] = 'true' if ADD_INDEX else 'false'

if (SAVE == 'save') or USE_DRIVE_CONFIG or (('://' not in URL) and ('/MyDrive/' in URL)):
    if not os.path.exists(DRIVE_MOUNT_PATH):
        drive.mount(DRIVE_MOUNT_PATH)
    if not os.path.exists(os.path.dirname(CONFIG_PATH)):
        os.makedirs(os.path.dirname(CONFIG_PATH))

if CLEAR_OUTPUT:
    print("🧹 Cleaning output and temp directories...")
    if os.path.exists(OUTPUT_DIR):
        shutil.rmtree(OUTPUT_DIR)
    if os.path.exists(TEMP_DIR):
        shutil.rmtree(TEMP_DIR)

if USE_DRIVE_CONFIG:
    if os.path.exists(CONFIG_PATH):
        print(f"📥 Loading settings from {CONFIG_PATH}...")
        with open(CONFIG_PATH, 'r') as f:
            for line in f:
                line = line.strip();
                if line and '=' in line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    if value:
                        os.environ[key] = value;
        print("✅ Settings applied.")
    print(f"💾 Saving settings to {CONFIG_PATH}...")
    with open(CONFIG_PATH, 'w') as f:
        for key, value in os.environ.items():
            if key.startswith('YT_') and value:
                f.write(f"{key}={value}\n")
    print("✅ Config updated!")

In [ ]:
# @title Cookies
%%writefile cookies.txt
# --- INSTRUCTIONS ---
# 1. Export cookies in Netscape format.
# 2. KEEP the "%%writefile cookies.txt" line at the top!
# 3. REPLACE everything below it with your pasted cookies.

In [ ]:
# @title Run Translation
! wget -q -N "https://raw.githubusercontent.com/alex2844/youtube-translate/main/ytranslate.sh"

if not COOKIES and os.path.exists("cookies.txt"):
    with open("cookies.txt") as f:
        if any(line.strip() and not line.strip().startswith('#') for line in f):
            COOKIES = "cookies.txt"

! YT_COOKIES="{COOKIES}" YT_OUTPUT_DIR="{OUTPUT_DIR}" YT_NO_CLEANUP="true" bash ./ytranslate.sh

In [ ]:
# @title Save / Download Results
from google.colab import files

save = os.environ['YT_SAVE']
if save != 'none':
    output_files = os.listdir(OUTPUT_DIR)
    if not output_files:
        print("No output files found. Did the script run successfully?")
    else:
        print(f"Found {len(output_files)} file(s). Processing...")
        for filename in output_files:
            src_path = os.path.join(OUTPUT_DIR, filename)
            if save == 'save':
                dest_path = os.path.join('/content/drive/MyDrive/ytranslate', filename)
                print(f"Copying to Drive: {filename}...")
                shutil.copy2(src_path, dest_path)
                print(f"✅ Saved: {dest_path}")
            elif save == 'download':
                print(f"Downloading: {filename}...")
                files.download(src_path)
        if save == 'save':
            print("\nAll files saved to '/ytranslate' folder on your Google Drive.")